In [80]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, time

In [44]:
exchange_info = {
    "LN": {
        "start": time(9, 30),
        "end": time(17, 30),
        "exchange_rate": 1.2853,
        "region": "WE",
        "name": "London"
    },
    "US": {
        "start": time(13, 30),
        "end": time(21, 30),
        "exchange_rate": 1,
        "region": "NA",
        "name": "United States"
    }
}

In [45]:
exchange_info

{'LN': {'start': datetime.time(9, 30),
  'end': datetime.time(17, 30),
  'exchange_rate': 1.2853,
  'region': 'WE',
  'name': 'London'},
 'US': {'start': datetime.time(13, 30),
  'end': datetime.time(21, 30),
  'exchange_rate': 1,
  'region': 'NA',
  'name': 'United States'}}

In [46]:
example_df = pd.DataFrame({
    "exchange": ["LN", "LN", "US", "LN", "US"],
    "start_time": ["09:34", "08:59", "10:44", "12:44", "9:34"],
    "end_time": ["09:39", "21:59", "10:48", "12:34", "11:34"],
    "price": [452, 145, 34, 12, 4],
    "avg_price": [45, 2, 45, 12, 5]
})

In [94]:
example_df["start_time"] = pd.to_datetime(example_df['start_time'], format='%H:%M').dt.time
example_df["end_time"] = pd.to_datetime(example_df['end_time'], format='%H:%M').dt.time

---

In [47]:
example_df.loc[5] = ["BZ", "05:44", "09:55", 43, 55]

In [98]:
def trade_within_hours(row):
    exchange = row["exchange"]
    
    if exchange in exchange_info.keys():
        trade_start = row["start_time"]
        trade_end = row["end_time"]
        
        open = exchange_info[exchange]["start"]
        close = exchange_info[exchange]["end"]
        
        if trade_start > open and trade_end < close:
            return True
        else: 
            return False
    else:
        print(f"exchange '{exchange}' not in dict")

In [100]:
example_df["within_hours"] = example_df.apply(trade_within_hours, axis=1)

exchange 'BZ' not in dict


In [107]:
def vectored_trade_within_hours():    
    start = {}
    end = {}

    for key, _ in exchange_info.items():
        start[key] = exchange_info[key]["start"]
        end[key] = exchange_info[key]["end"]
        
    start_vect = example_df["exchange"].map(start)
    end_vect = example_df["exchange"].map(end)

    example_df["within_hours_vectored"] = np.where((example_df["start_time"] > start_vect) & (example_df["end_time"] < end_vect), True, False) 

In [108]:
vectored_trade_within_hours()

In [109]:
example_df

,exchange,start_time,end_time,price,avg_price,within_hours,within_hours_vectored
0,LN,09:34:00,09:39:00,452,45,True,True
1,LN,08:59:00,21:59:00,145,2,False,False
2,US,10:44:00,10:48:00,34,45,False,False
3,LN,12:44:00,12:34:00,12,12,True,True
4,US,09:34:00,11:34:00,4,5,False,False
5,BZ,05:44:00,09:55:00,43,55,None,False


In [129]:
start = {}
end = {}
for key, _ in exchange_info.items():
    start[key] = exchange_info[key]["start"]
    end[key] = exchange_info[key]["end"]
    
start_vect = example_df["exchange"].map(start)
end_vect = example_df["exchange"].map(end)

((example_df["start_time"] > start_vect) & (example_df["start_time"] < end_vect))

0     True
1    False
2    False
3     True
4    False
5    False
dtype: bool

In [154]:
def vectored_trade_within_hours():    
    start = {}
    end = {}

    for key, _ in exchange_info.items():
        start[key] = exchange_info[key]["start"]
        end[key] = exchange_info[key]["end"]
        
    start_vect = example_df["exchange"].map(start)
    end_vect = example_df["exchange"].map(end)
    
    conditions = [
        ((example_df["start_time"] > start_vect) & (example_df["end_time"] < end_vect)),
        ((example_df["start_time"] < start_vect) & (example_df["end_time"] > end_vect)),
        ((example_df["start_time"] < start_vect) & (example_df["end_time"] < start_vect)),
        ((example_df["start_time"] < start_vect) & (example_df["end_time"] < end_vect))
    ]
    
    choices = ["inside", "outside",  "outside", "overlap"]
    
    example_df["test_vec"] = np.select(conditions, choices, default=np.nan)

In [155]:
vectored_trade_within_hours()

In [156]:
example_df

,exchange,start_time,end_time,price,avg_price,within_hours,within_hours_vectored,test_vec
0,LN,09:34:00,09:39:00,452,45,True,True,inside
1,LN,08:59:00,21:59:00,145,2,False,False,outside
2,US,10:44:00,10:48:00,34,45,False,False,outside
3,LN,12:44:00,12:34:00,12,12,True,True,inside
4,US,09:34:00,11:34:00,4,5,False,False,outside
5,BZ,05:44:00,09:55:00,43,55,None,False,nan


---

In [9]:
exchange_map = {}

for key, value in exchange_info.items():
    exchange_map[key] = value["region"]
    
exchange_map

{'LN': 'WE', 'US': 'NA'}

In [10]:
example_df["region"] = example_df["exchange"].map(exchange_map)

example_df

,exchange,start_time,end_time,price,avg_price,within_hours,region
0,LN,09:34,09:39,452,45,True,WE
1,LN,08:59,21:59,145,2,False,WE
2,US,10:44,10:48,34,45,False,NA
3,LN,12:44,12:34,12,12,True,WE
4,US,9:34,11:34,4,5,False,NA
5,BZ,05:44,09:55,43,55,None,NaN


---

In [11]:
exchange_rate = {}

for key, value in exchange_info.items():
    exchange_rate[key] = value["exchange_rate"]
    
exchange_rate

{'LN': 1.2853, 'US': 1}

In [12]:
pre_multi = example_df.copy()

In [13]:
price_columns = ["price", "avg_price"]

for price_column in price_columns:
    example_df[price_column] = example_df["exchange"].map(exchange_rate) * example_df[price_column]

In [14]:
exchange_rate

{'LN': 1.2853, 'US': 1}

In [15]:
pre_multi

,exchange,start_time,end_time,price,avg_price,within_hours,region
0,LN,09:34,09:39,452,45,True,WE
1,LN,08:59,21:59,145,2,False,WE
2,US,10:44,10:48,34,45,False,NA
3,LN,12:44,12:34,12,12,True,WE
4,US,9:34,11:34,4,5,False,NA
5,BZ,05:44,09:55,43,55,None,NaN


In [16]:
example_df

,exchange,start_time,end_time,price,avg_price,within_hours,region
0,LN,09:34,09:39,580.9556,57.8385,True,WE
1,LN,08:59,21:59,186.3685,2.5706,False,WE
2,US,10:44,10:48,34.0000,45.0000,False,NA
3,LN,12:44,12:34,15.4236,15.4236,True,WE
4,US,9:34,11:34,4.0000,5.0000,False,NA
5,BZ,05:44,09:55,NaN,NaN,None,NaN


## Action Plan for 11/10/2023
- Create .py file containing a dictionary which holds exchange information
- Use the .apply() function to map all of the regions, exchange rates, and whether the trade was within the opening/close hours
- Investigate the use of trade_value and qty_executed as the weights for analysis

In [ ]:
exchange_info = {}

for index, row in df.itterow():
    if row["PrimExch"] not in exchange_info:
        exchange_info[row["PrimExch"]] = {
            "open": row["open"],
            "close": row["open"],
            "name": row["name"],
            "region": row["region"],
            "country": row["country"]
        }